# SM from IRRmodel into WCM, both with PSO on KGE

The v3 code is an improvement of the v2 code to provide hourly calibration of IRRmodel and additional masking of outliers/unphysical values (such as very low backscatter at times of irrigation/rain)

# Dependencies, forward and backwards model

In [1]:
# Base
import os
import re
import time
import math
import numpy as np
import pandas as pd
import datetime as dtt

# Analysis
import pyswarms as ps
from scipy import special as sp
from scipy.optimize import curve_fit
from numpy.polynomial import Polynomial
from scipy.signal import savgol_filter as sfilter

# Geospatial
# import fiona
import xarray as xr
import hydroeval as he
# import geopandas as gpd
# from maps_original import *

# Graphics
import seaborn as sns
import matplotlib as mplt
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from pyswarms.utils.plotters import plot_cost_history
from pyswarms.utils.functions.single_obj import sphere

In [2]:
# Ausiliary functions

#-----------------------------------------------------------------------------

def lin_db(x):
    return 10*np.log10(x)

def db_lin(x):
    return 10**(x/10)

def norm(x):
    return (x-np.min(x))/(np.max(x)-np.min(x))

def linear(x,a,b):
    return a+b*x

def bias(obs, sim):
    if len(obs)==len(sim):
        return np.mean(obs-sim)
    else: raise ValueError(
        f'obs and sim must have same first dimension, but have shapes {np.shape(obs)} and {np.shape(sim)}')
    
def timeseries(dates, data):
    """Returns a matrix (list of type(dates,data)) in the format [dates,data]"""
    
    if len(dates)==len(data):
        return [[dates[i],data[i]] for i in range(len(dates))]
    else: raise ValueError(
        f'dates and data must have same first dimension, but have shapes {np.shape(dates)} and {np.shape(data)}')

In [39]:
def IRR_WCM(PAR, inputs, user_in):
    """Irrigation model and WCM integration.
    
    Based on minimization of KGE between observed and simulated
    backscattering via PSO (pyswarm) optimization.
    Version with 8 parameters to calibrate.
    
    Inputs
    ----------
    - PAR: initial guess values for parameters to calibrate
        PAR = [A, B, C, D, W_0, W_max, S_fc, S_w, rho_st, Kc]
    - inputs: input quantities for calibration,
        [d, d_sat, P, IRRobs, EPOT, WWobs, LAI, t_deg, obs]
    
    Return
    -------
    KGE from hydroeval between sigma0 observed and simulated.
    
    """

    # User input
    irri = user_in
    
    # Unpack inputs
    A, B, C, D, W_0, W_max, S_fc, S_w, rho_st, Kc = PAR
    d, d_sat, P, IRRobs, EPOT, WWobs, LAI, t_deg, obs = inputs
    
    #S_fc = 0.46 # hardcoded
    #S_w = 0.08  # hardcoded
    W_fc = S_fc*W_max # water content at field capacity
    W_w  = S_w*W_max # water content at wilting point
    theta = t_deg*np.pi/180. # angle of observation
    
    
    if irri==True: IRR = [0]*len(d) # daily, water content
    else: IRR = IRRobs
    
    Ks = [0]*len(d) # daily, water stress coefficient
    rho = [0]*len(d) # daily, depletion fraction
    PS = [0]*len(d) # daily, deep percolation
    W = [0]*len(d) # daily, water content
    W[0] = W_0*W_max
    
    for t in [i+1 for i in range(len(d)-1)]:
        rho[t]=rho_st+0.04*(5-Kc*EPOT[t])
        if W[t-1]>=(1-rho[t])*W_fc:
            Ks[t]=1
        elif (W[t-1]>W_w)and(W[t-1]<(1-rho[t])*W_fc):
            Ks[t]=float(W[t-1]-W_w)/((1-rho[t])*(W_fc-W_w))
        else: Ks[t]=0
        
        DOY=d[t].dayofyear
        
        # Irrigation estimate (for summer season only)
        # Irrigation is estimated as the amount of water needed at the day before
        # to take water content up to field capacity
        if np.logical_and(DOY>134,DOY<235): # summer season
            if irri==True:
                if W[t-1]<=(1-rho[t])*W_fc: IRR[t]=W_fc-W[t-1]
        
        # Water balance
        W[t]=W[t-1]+P[t]+IRR[t]-EPOT[t]*Kc*Ks[t]
        
        # Computation of deep percolation (water above field capacity)
        if W[t]>W_fc:
            PS[t]=W[t]-W_fc
            W[t]=W_fc
            
    WW=np.array(W)/W_max   
    WWsat = pd.DataFrame(timeseries(d,WW)).set_index(0).loc[d_sat][1].values
    
    sig0s = db_lin(C+D*WWsat) # define bare soil backscatter [lin]
    T2 = np.exp((-2*B*LAI)/np.cos(theta)) # two-way attenuation from the vegetation layer
    sig0v = A*LAI*np.cos(theta)*(1-T2) # define backscatter from the vegetation
    sig0_lin = T2*sig0s+sig0v
    sig0=lin_db(sig0_lin) # from linear scale to dB
        
    OUT=he.evaluator(he.kge, sig0, obs) # OUT is kge, r, alpha, beta
    KGE=OUT[0,:];

    return [WW,IRR,sig0,KGE]

#-----------------------------------------------------------------------------

def pso_calib_irri(PAR):
    """Ausiliary function for PSO optimization"""
    global inputs
    global irri
    n_particles = PAR.shape[0]
    err = np.zeros(n_particles)
    for i in range(n_particles):
        WW,IRR,sig0,KGE = IRR_WCM(PAR[i], inputs, irri)
        err[i] = 1 - KGE
    return err

# IRRmodel+WCM run

In [18]:
print('Starting...\n'+
      '#-------------------------------------------------------------\n'+
      'Use of satellite-derived SM is provided for comparison, not calibration.\n')

#-----------------------------------------------------------------------------
# Data reading
#-----------------------------------------------------------------------------

filename = f'irr_obs_'

# Field data from TEST_SITE
namesite = 'ITALY_BUDRIO'
siteID = '5'
namefig = namesite+'_'+siteID
sitedf = xr.open_dataset(f'IRRmodel\TEST_SITE\TEST_SITE_{namesite}.nc', engine='netcdf4').to_dataframe(); sitedf

# Satellite and veg data (S-1) from cleaned data table
df = pd.read_csv('Data\data_in.csv', delimiter='\t', index_col=0);
df.Date = df.Date.apply(lambda x : pd.to_datetime(x))
df = df.set_index('Date')

# Budrio field data from Platinum tables
platinum = pd.ExcelFile('Data\Platinum_Budrio.xlsx', engine='openpyxl')
platinum = platinum.parse('2017_1h')
platinum['Ora_1'] = pd.to_datetime(platinum['Ora'].astype('str')).apply(lambda x: x.time())
platinum['Data_1'] = pd.to_datetime(platinum['Data'].astype('str')).apply(lambda x: x.date())
platinum['Date'] = platinum.apply(lambda r : dtt.datetime.combine(r['Data_1'],r['Ora_1']),1)
platinum = platinum.drop(['ID', 'Data_1', 'Ora_1', '214Pb[cps]'],axis=1)
#platinum_cut = pd.concat([platinum.loc[(platinum.Data<'2017-05-14')],platinum.loc[(platinum.Data>'2017-06-16')]])


# Dates
# D_0 = sitedf.Time_days; D_1 = platinum['Data']
# set1 = {x for x in D_0}; set2 = {x for x in D_1}
# d = np.sort(np.array([*set1.intersection(set2)])) # dates, full
d = platinum.Date

d_sat = [pd.Timestamp(df.index[i]) for i in range(len(df.index))]
set1 = {x for x in d}; set2 = {x for x in d_sat}
d_sat = np.sort(np.array([*set1.intersection(set2)])) # dates, passage of sat

h_sat = [pd.Timestamp(df.DateTime_sat[i]) for i in range(len(df.index))]
set1 = {x for x in d}; set2 = {x for x in h_sat}
h_sat = np.sort(np.array([*set1.intersection(set2)])) # date and hour, passage of sat


# Backscattering
# t_deg = df['Angle[°]'].values
t_deg = [np.mean(df.loc[df.Orb==95]['Angle[°]'].values) for i in range(len(df.Orb.values))]
obs = df.loc[d_sat]['VV_norm[dB]'].values
obs_VH = df.loc[d_sat]['VH_norm[dB]'].values
rt1 = df.loc[d_sat]['RT1[-]']


# Vegetation indexes
LAI = df.loc[d_sat]['LAI[m2/m2]'].values
cr = (db_lin(obs_VH)/db_lin(obs))

Starting...
#-------------------------------------------------------------
Use of satellite-derived SM is provided for comparison, not calibration.



In [20]:
# -----------------------------------------------------------------------------
# User input
#-----------------------------------------------------------------------------

params = []; norma = ''
#opt_time = input('Daily aggregated dataset or hourly? [d/h]')
nrun = int(input('Number of runs? (10 is min to study distribution of parameters.) '))
n_particles = int(input('Number of particles: '))
n_step = int(input('Number of optimization steps: '))
verbose = True if input('Verbose? [[]/(any)]')=='' else False
optim = input('Global or Local PSO optimizer? [[global]/local] ')
if optim=='local': norma = 1 if input('Which norm? [l1/l2] ')=='l1' else 2
irri = True if input('Do you want to calibrate irrigation as well as soil moisture? Answer yes will treat irrigation observations as a benchmark for model performance only. [[]/(any)')=='' else False

opt_time='h' #hardcoded for now
if opt_time=='d':
    P = platinum.resample('1D',on='Date').sum().loc[d]['Pioggia[mm]'].values
    EPOT = sitedf.set_index('Time_days').loc[d][f'PET_{siteID}'].values # potential evapotranspiration (measured)
    IRRobs = platinum.resample('1D',on='Date').sum().loc[d]['Irrigazione[mm]'].values
    Wobs_gap = platinum.resample('1D',on='Date').mean().loc[d]['SWC[m3/m3]'].values
    Wobs = platinum.resample('1D',on='Date').mean().loc[d]['SWC[m3/m3]'].interpolate(method='linear').values
    WWobs_gap = (Wobs_gap-np.min(Wobs))/(np.max(Wobs)-np.min(Wobs))
    WWobs = norm(Wobs)
elif opt_time=='h':
    d_sat = h_sat
    df1 = pd.DataFrame(timeseries(sitedf.Time_days, sitedf[f'PET_{siteID}'])).rename(columns={0:'Data', 1:'EPOT'}); epotdf=pd.merge(right=df1, left=platinum, on='Data', how='left')
    EPOT = epotdf.EPOT.values
    P = platinum['Pioggia[mm]'].values
    IRRobs = platinum['Irrigazione[mm]'].values
    Wobs_gap = platinum['SWC[m3/m3]'].values
    Wobs = platinum['SWC[m3/m3]'].values
    WWobs_gap = (Wobs_gap-np.min(Wobs))/(np.max(Wobs)-np.min(Wobs))
    WWobs = norm(Wobs)

index = input('Please provide name of vegetation index to use as input. [Options: LAI, cr, vh]')
if index=='LAI': veg=LAI; label_veg='LAI[-]'
elif index=='cr': veg=cr; label_veg=r'$\sigma^0$ cross ratio VH/VV [-]'
elif index=='vh': veg=db_lin(obs_VH); label_veg=r'$\sigma^0$ [VH] [dB]'
else: raise NameError('Please select one of the available options')

Number of runs? (10 is min to study distribution of parameters.)  1
Number of particles:  10
Number of optimization steps:  100
Verbose? [[]/(any)] 
Global or Local PSO optimizer? [[global]/local]  
Do you want to calibrate irrigation as well as soil moisture? Answer yes will treat irrigation observations as a benchmark for model performance only. [[]/(any) 
Please provide name of vegetation index to use as input. [Options: LAI, cr, vh] LAI


In [36]:
d = [pd.Timestamp(x) for x in d]

In [40]:
#-----------------------------------------------------------------------------
# Calibration
#-----------------------------------------------------------------------------

print('Starting calibration...\n'+
      '#-------------------------------------------------------------\n')

A=0.1; B=0.1; C=-10; D=3 # guess params for WCM
W_0=      0.1; # [-] water content, initial [m^3/m^3]
W_max=    80;  # [mm] water content, maximum (not normalized)
S_fc=     0.46;
S_w=      0.08;
rho_st=   0.2; # [-] crop specific depletion fraction
Kc=       0.4; # [-] crop specific coefficient

inputs = [d, d_sat, P, IRRobs, EPOT, WWobs, veg, t_deg[0], obs]
PAR = [A, B, C, D, W_0, W_max, S_fc, S_w, rho_st, Kc];
# PAR = [A, B, C, D, W_0, W_max, rho_st, Kc];

bounds = (
    np.array([ .01, .01, -20, 1, 0.1, 10, .01, .01, .5, .3]), # # low
    np.array([  1,  1,   -5, 15, 0.9, 500, .95, .45, .8, .6]), # # up
)
# bounds = (
#     np.array([ .01,.01,-20, 0, 0.1, 10, .5, .3]), #.01, .01, # low
#     np.array([  1, 1, 0, 15,  0.9, 150, .8, .6]), #.95, .45, # up
# )
# bounds = (
#     np.array([  0.5,  50, .80, .01, .1, .3]),# low
#     np.array([  0.9, 120, .95, .45, .7, .5]) # up
# )

#-----------------------------------------------------------------------------
from pyswarms.backend.handlers import OptionsHandler

for i in range(int(nrun)):
    if (optim=='global')or(optim==''):
        # options = {'c1': 0.5, 'c2': 0.9, 'w': 0.6}
        options = {'c1': 0.5, 'c2': 0.9, 'w': 10}
        optimizer = ps.single.GlobalBestPSO(n_particles=int(n_particles), dimensions=len(PAR),
                                            options=options, bounds=bounds, oh_strategy={"w":'exp_decay', 'c1':'lin_variation', 'c2':'lin_variation'})
    elif optim=='local':
        options = {'c1': 0.4, 'c2': 0.4, 'w': 0.8, 'k':int(0.1*n_particles), 'p':norma }
        optimizer = ps.single.LocalBestPSO(n_particles=n_particles, dimensions=len(PAR), options=options, bounds=bounds)
    else: raise NameError('Please provide an accepted option.')
    cost, PARn = optimizer.optimize(pso_calib_irri, n_step, verbose=verbose)#, **PAR)
    params.append(PARn)
    i+=1

#-----------------------------------------------------------------------------
# Model validation
#-----------------------------------------------------------------------------
WW,IRR,sigma0,KGE = IRR_WCM(PARn, inputs, irri)
print('PAR = [A, B, C, D, W_0, W_max, rho_st, Kc]\n', PARn)
timestr = time.strftime("%y%m%d-%H%M%S")

#-----------------------------------------------------------------------------
# Study distribution of parameters
#-----------------------------------------------------------------------------
matrix = np.array([ np.array([ params[i][j] for i in range(len(params)) ])
        for j in range(len(params[0])) ])

if (int(nrun)>9)and(input('Plot parameters distributions? [y/n]')=='y'):
    for i in range(len(matrix)):
        plt.hist(matrix[i])
        plt.show()

#-----------------------------------------------------------------------------
# Figures
#-----------------------------------------------------------------------------

opt_save_plot = False# True if input('Save simulated VS observed soil moisture? [[]/(any)]')=='' else False

#-----------------------------------------------------------------------------
# Model performance on SM and plot
#-----------------------------------------------------------------------------
RMSE=np.nanmean((WW-WWobs)**2)**0.5; print('RMSE =', RMSE)
NS=1-np.nansum((WW-WWobs)**2)/np.nansum((WWobs-np.nanmean(WWobs))**2); print('NS =', NS)
NS_radQ=1-np.nansum((np.sqrt(WW+0.00001)-np.sqrt(WWobs+0.00001))**2)/np.nansum((np.sqrt(WWobs+0.00001)-np.nanmean(np.sqrt(WWobs+0.00001)))**2)
NS_lnQ=1-np.nansum((np.log(WW+0.0001)-np.log(WWobs+0.0001))**2)/np.nansum((np.log(WWobs+0.0001)-np.nanmean(np.log(WWobs+0.0001)))**2)
NS_lnQ=NS_lnQ.real; # print(NS_lnQ) 
NS_radQ=NS_radQ.real; # print(NS_radQ)

# R coefficient score
WWmatrix = np.array( [ [WW[i], WWobs[i]] for i in range(len(WW)) if not np.isnan(WWobs[i]) ] )
R=np.corrcoef(WWmatrix,rowvar=False)[0][1]; print('R (WW vs WWobs) =', R)
B=bias(np.array([e[0] for e in WWmatrix]), np.array([e[1] for e in WWmatrix]))
IRRmatrix = np.array( [ [IRR[i], IRRobs[i]] for i in range(len(IRR)) if not np.isnan(IRRobs[i]) ] )
R_IRR=np.corrcoef(IRRmatrix,rowvar=False)[0][1]; print('R_IRR (IRR vs IRRobs)=', R_IRR)
B_IRR=bias(np.array([e[0] for e in IRRmatrix]), np.array([e[1] for e in IRRmatrix]))

fig, ax = plt.subplots(2, 1,constrained_layout=True,figsize=(12, 6), sharex=True,dpi=300,)

title=r'$\theta_{obs}\;VS\;\theta_{sim}$ - '+\
    f'sumIRRobs={np.sum(IRRobs):.2f}, '+\
    f'sumIRRsim={np.sum(IRR):.2f}, '+\
    f'R_SM={R:.2f}, R_IRR={R_IRR:.2f}, '+\
    f'bias_SM={B:.2f}, bias_IRR={B_IRR:.2f}'

ax[0].set_xlim(xmin=d[0], xmax=d[len(d)-1])
ax[0].plot(d, WW, c='tab:red', label=r'$\theta_{sim}$')
ax[0].plot(d, WWobs, c='tab:blue', #label=r'$\theta_{obs,interp}$',
               linestyle='-', alpha=.4, zorder=-1)
ax[0].plot(d, WWobs_gap, c='tab:blue', label=r'$\theta_{obs}$', alpha=.5, zorder=2)
ax[0].legend(loc='upper right')
ax[0].set_title(title)
ax[0].set_ylabel('Relative SM [-]')

ax[1].plot(d, IRR, c='r', label=r'$IRR_{sim}$')
ax[1].plot(d, IRRobs, c='b', label=r'$IRR_{obs}$')
ax[1].plot(d, P, c='gray', label=r'rainfall')
ax[1].legend(loc='upper right')
ax[1].set_ylabel('Irrigation and rain [mm]')

name=''
if opt_save_plot==True:
    optim_choice = 'glo' if (optim=='')or(optim=='global') else 'local'
    name = filename+timestr+f'_{n_particles}_{n_step}_{optim_choice}_{norma}'
    plt.savefig('Plot\\'+name+'.png')
    plt.show()
    plt.close()
    cost_history = optimizer.cost_history
    plot_cost_history(cost_history)
    plt.savefig('Plot\\'+name+'_cost'+'.png')

#-----------------------------------------------------------------------------
# Model performance on sigma0 and plot
#-----------------------------------------------------------------------------
q1 = sigma0; q2 = obs; times = d_sat
label1 = r'$\sigma^0_{sim}$'; label2=r'$\sigma^0_{obs}$'; labely=r'$\sigma^0$[dB]'
marker='o'; linestyle='-'

NS=1-np.nansum((q1-q2)**2)/np.nansum((q2-np.nanmean(q2))**2)
NS_radQ=1-np.nansum((np.sqrt(q1+0.00001)-np.sqrt(q2+0.00001))**2)/np.nansum((np.sqrt(q2+0.00001)-np.nanmean(np.sqrt(q2+0.00001)))**2)
NS_lnQ=1-np.nansum((np.log(q1+0.0001)-np.log(q2+0.0001))**2)/np.nansum((np.log(q2+0.0001)-np.nanmean(np.log(q2+0.0001)))**2)
NS_lnQ=NS_lnQ.real; # print(NS_lnQ) 
NS_radQ=NS_radQ.real; # print(NS_radQ)

RMSE=np.mean((q1-q2)**2)**0.5; print('RMSE =', RMSE)
R=np.corrcoef(q1,q2)[0][1]; print('R=', R)
B=bias(q1,q2); print('bias =', B)

fig, ax = plt.subplots(2, 1,constrained_layout=True,figsize=(12, 6), sharex=True,dpi=300,)

title=f'{label1} VS {label2} - R={R:.2f}, bias={B:.2f}'

ax[0].set_xlim(xmin=times[0], xmax=times[len(times)-1])
ax[0].plot(times, q1, c='tab:red', label=label1,
           linestyle=linestyle, marker=marker, )#alpha=.4, zorder=-1)
ax[0].plot(times, q2, c='tab:blue', label=label2,
           linestyle=linestyle, marker=marker, alpha=.4, zorder=-1)
ax[0].legend(loc='upper right')
ax[0].set_title(title)
ax[0].set_ylabel(labely)

ax[1].plot(d, IRR, c='r', label=r'$IRR_{sim}$')
ax[1].plot(d, IRRobs, c='b', label=r'$IRR_{obs}$')
ax[1].plot(d, P, c='gray', label=r'rainfall')
ax[1].legend(loc='upper right')
ax[1].set_ylabel('Irrigation and rain [mm]')

2022-11-26 19:52:44,045 - pyswarms.single.global_best - INFO - Optimize for 100 iters with {'c1': 0.5, 'c2': 0.9, 'w': 10}


Starting calibration...
#-------------------------------------------------------------



pyswarms.single.global_best:   0%|                                                                |0/100, best_cost=inf


ValueError: operands could not be broadcast together with shapes (0,) (10,10) 